In [1]:
# %cd ..

/home/hnguyen/nghia_branch/ourproject/equiRL


In [2]:
# !. ./prepare_1.0.sh

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import sys
from torchsummary import summary
import torchvision.models as model_pretrain

In [31]:
class TanhWrappedDistribution(torch.distributions.Distribution):
    """
    Class that wraps another valid torch distribution, such that sampled values from the base distribution are
    passed through a tanh layer. The corresponding (log) probabilities are also modified accordingly.
    Tanh Normal distribution - adapted from rlkit and CQL codebase
    (https://github.com/aviralkumar2907/CQL/blob/d67dbe9cf5d2b96e3b462b6146f249b3d6569796/d4rl/rlkit/torch/distributions.py#L6).
    """
    def __init__(self, base_dist, scale=1.0, epsilon=1e-6):
        """
        Args:
            base_dist (Distribution): Distribution to wrap with tanh output
            scale (float): Scale of output
            epsilon (float): Numerical stability epsilon when computing log-prob.
        """
        self.base_dist = base_dist
        self.scale = scale
        self.tanh_epsilon = epsilon
        super(TanhWrappedDistribution, self).__init__()

    def log_prob(self, value, pre_tanh_value=None):
        """
        Args:
            value (torch.Tensor): some tensor to compute log probabilities for
            pre_tanh_value: If specified, will not calculate atanh manually from @value. More numerically stable
        """
        value = value / self.scale
        if pre_tanh_value is None:
            one_plus_x = (1. + value).clamp(min=self.tanh_epsilon)
            one_minus_x = (1. - value).clamp(min=self.tanh_epsilon)
            pre_tanh_value = 0.5 * torch.log(one_plus_x / one_minus_x)
        lp = self.base_dist.log_prob(pre_tanh_value)
        tanh_lp = torch.log(1 - value * value + self.tanh_epsilon)
        # In case the base dist already sums up the log probs, make sure we do the same
        return lp - tanh_lp if len(lp.shape) == len(tanh_lp.shape) else lp - tanh_lp.sum(-1)
    def sample(self, sample_shape=torch.Size(), return_pretanh_value=False):
        """
        Gradients will and should *not* pass through this operation.
        See https://github.com/pytorch/pytorch/issues/4620 for discussion.
        """
        z = self.base_dist.sample(sample_shape=sample_shape).detach()

        if return_pretanh_value:
            return torch.tanh(z) * self.scale, z
        else:
            return torch.tanh(z) * self.scale

    def rsample(self, sample_shape=torch.Size(), return_pretanh_value=False):
        """
        Sampling in the reparameterization case - for differentiable samples.
        """
        z = self.base_dist.rsample(sample_shape=sample_shape)

        if return_pretanh_value:
            return torch.tanh(z) * self.scale, z
        else:
            return torch.tanh(z) * self.scale

In [32]:
class BC_RNN_GMM_actor(nn.Module):
    def __init__(self, obs_shape = (1,128,128), device = "cpu", node = 5):
        super(BC_RNN_GMM_actor, self).__init__()
        self.channel = obs_shape[0]
        self.device = device
        self.node = node
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=self.channel, out_channels=32, kernel_size=3, stride =1, padding=1),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride =1, padding = 1),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding = 1),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size = 3, stride =1, padding = 1),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(256*8*8, 512)
        self.fc2 = nn.Linear(512,128)
        self.fc3 = nn.Linear(128,32)

        self.per_step_mean = nn.Linear(200, node*8)
        self.per_step_std = nn.Linear(200, node*8)
        self.per_step_logit = nn.Linear(200, node)

        self.rnn = nn.LSTM(32, 200 , 2, batch_first=True)

        self.relu = nn.ReLU()
    def forward(self, x, init_state = None, train = True):
        # Process the input image with the CNN
        length = x.shape[1]
        x = x.view(x.size(0)*x.size(1),x.size(2),x.size(3),x.size(4))
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        
        # Process the CNN output with the LSTM
        x = self.relu(self.fc1(x))

        x = self.relu(self.fc2(x))

        x = self.fc3(x)

        x = x.view(-1,length,x.size(1))
        
        if init_state is None:
            h0 = torch.zeros(2,x.size(0),200).to(self.device)
            c0 = torch.zeros(2,x.size(0),200).to(self.device)
            init_state = (h0,c0)
        out, (hn,cn) = self.rnn(x, init_state)

        out = out.contiguous().view(-1,200)

        mean = self.per_step_mean(out)

        mean = mean.view(-1, length, self.node, 8)

        std = self.per_step_std(out)
        
        std = std.view(-1, length, self.node, 8)

        logits = self.per_step_logit(out)

        logits = logits.view(-1, length, self.node)


        if not train:
            std = torch.ones_like(mean) * 1e-4
        else:
            std = torch.nn.functional.softplus(std) + 1e-4

        component_distribution = torch.distributions.Normal(loc=mean, scale=std)
        component_distribution = torch.distributions.Independent(component_distribution, 1)
        mixture_distribution = torch.distributions.Categorical(logits=logits)

        dists = torch.distributions.MixtureSameFamily(
            mixture_distribution=mixture_distribution,
            component_distribution=component_distribution,
        )

        dists = TanhWrappedDistribution(base_dist=dists, scale=1.)

        return dists, hn, cn

In [33]:
model = BC_RNN_GMM_actor((1,128,128), device = "cpu")

In [34]:
input = torch.randn((3,10,1,128,128))

In [35]:
dist, hn, cn = model(input)

/home/hnguyen/miniconda3/envs/softgym/lib/python3.8/site-packages/torch/distributions/distribution.py:45: UserWarning: <class '__main__.TanhWrappedDistribution'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [41]:
actions = dist.sample()

In [44]:
actions.max()

tensor(0.9857)

In [17]:
class ResNet50Gray(nn.Module):
    def __init__(self):
        super(ResNet50Gray, self).__init__()
        # Load the pre-trained ResNet-50 model
        resnet = model_pretrain.resnet50(pretrained=True)
        # Modify the first convolutional layer to accept 1 input channel instead of 3
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Copy the weights from the original RGB channel to all 3 channels
        self.conv1.weight.data[:, :3, :, :] = resnet.conv1.weight.data.clone()
        self.conv1.weight.data[:, 3:, :, :] = self.conv1.weight.data[:, :1, :, :]
        # Use the rest of the pre-trained ResNet-50 model
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        self.avgpool = resnet.avgpool
        self.fc = resnet.fc

    def forward(self, x):
        # Convert grayscale images to RGB format
        x = x.repeat(1, 3, 1, 1)
        # Use the modified first convolutional layer
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [18]:
resnet = ResNet50Gray()

/home/hnguyen/miniconda3/envs/softgym/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hnguyen/miniconda3/envs/softgym/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: The expanded size of the tensor (1) must match the existing size (3) at non-singleton dimension 1.  Target sizes: [64, 1, 7, 7].  Tensor sizes: [64, 3, 7, 7]

In [2]:
class RNN_actor(nn.Module):
    def __init__(self, input_shape = (1,128,128), device = "cpu"):
        
        super(RNN_actor, self).__init__()
        
        # Define the CNN
        # self.length = input_shape[0]
        self.channel = input_shape[0]
        self.device = device
        # self.length_video = length_video
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=self.channel, out_channels=32, kernel_size=3, stride =1, padding=1),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride =1, padding = 1),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding = 1),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(128*16*16, 512)
        self.rnn = nn.RNN(512, 64, 1, batch_first=True)
        self.fc2 = nn.Linear(64, 8)
        
    def forward(self, x):
        # Process the input image with the CNN
        length = x.shape[1]
        x = x.view(x.size(0)*x.size(1),x.size(2),x.size(3),x.size(4))
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        
        # Process the CNN output with the LSTM
        x = self.fc1(x)

        x = x.view(-1,length,x.size(1))
        
        h0 = torch.zeros(1, x.size(0), 64).to(self.device)

        out, _ = self.rnn(x, h0)

        out = self.fc2(out)
        # Compute the output
        return out


In [27]:
class BC_RNN_actor(nn.Module):
    def __init__(self, obs_shape = (1,128,128), device = "cpu"):
        
        super(BC_RNN_actor, self).__init__()
        
        # Define the CNN
        # self.length = input_shape[0]
        self.channel = obs_shape[0]
        self.device = device
        # self.picker_shape = picker_shape
        # self.length_video = length_video
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=self.channel, out_channels=32, kernel_size=3, stride =1, padding=1),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride =1, padding = 1),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding = 1),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size = 3, stride =1, padding = 1),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(256*8*8, 512)
        self.fc2 = nn.Linear(512,128)
        self.fc3 = nn.Linear(128,32)
        self.rnn = nn.LSTM(32, 200 , 2, batch_first=True)
        self.per_step = nn.Linear(200, 8)
        self.relu = nn.ReLU()
    def forward(self, x):
        # Process the input image with the CNN
        length = x.shape[1]
        x = x.view(x.size(0)*x.size(1),x.size(2),x.size(3),x.size(4))
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        
        # Process the CNN output with the LSTM
        x = self.relu(self.fc1(x))

        x = self.relu(self.fc2(x))

        x = self.fc3(x)

        x = x.view(-1,length,x.size(1))
        

        out, _ = self.rnn(x)

        out = out.contiguous().view(-1,200)

        out = self.per_step(out)

        out = out.view(-1, length, out.size(1))

        out = torch.tanh(out)

        # Compute the output
        return out


In [28]:
model = BC_RNN_actor((3,128,128))

In [29]:
x = torch.randn((3,30,3,128,128))

In [24]:
x = x.view(1,1,*x.shape)

In [30]:
out = model(x)

In [26]:
out.contiguous().view(-1,200).shape

torch.Size([90, 200])

In [26]:
out = out.squeeze()

In [32]:
out.cpu().detach().numpy()

array([ 0.1168578 , -0.07547387,  0.09831032,  0.09051739, -0.15409046,
       -0.03680436,  0.1875623 ,  0.05464403], dtype=float32)

In [15]:
summary(model,input_size = (1,3,128,128), device = "cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             896
         MaxPool2d-2           [-1, 32, 64, 64]               0
            Conv2d-3           [-1, 64, 64, 64]          18,496
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5          [-1, 128, 32, 32]          73,856
         MaxPool2d-6          [-1, 128, 16, 16]               0
            Linear-7                  [-1, 512]      16,777,728
               RNN-8  [[-1, 1, 64], [-1, 2, 64]]               0
            Linear-9                 [-1, 1, 8]             520
Total params: 16,871,496
Trainable params: 16,871,496
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 8.69
Params size (MB): 64.36
Estimated Total Size (MB): 73.24
----------------------------------